# Using the Yelp API - Codealong

## Introduction

Now that we've discussed HTTP requests and OAuth, it's time to practice applying those skills to a production level API. In this codealong, we'll take you through the process of signing up for an OAuth token and then using that to make requests to the Yelp API!

## Objectives

You will be able to:

* Make requests using OAuth
* Use the JSON module to load and parse JSON documents
* Convert JSON to a pandas dataframe

## Generating Access Tokens

As discussed, in order to use many APIs, one needs to use OAuth which requires an access token. As such, our first step will be to generate this login information so that we can start making some requests.  

With that, let's go grab an access token from an API site and make some API calls!
Point your browser over to this [yelp page](https://www.yelp.com/developers/v3/manage_app) and start creating an app in order to obtain and API access token:

![](./images/yelp_app.png)


You can either sign in to an existing Yelp account or create a new one if needed.

On the page you see above, simply fill out some sample information such as "Flatiron Edu API Example" for the app name, or whatever floats your boat. Afterward, you should be presented with an API key that you can use to make requests!

With that, let's set up our authentication tokens so that we can start making some API calls!

### Should I publicly share my passwords on Github?

When using an API that requires an API key and password you should **NEVER** hardcode theses values into your main file. When you upload your project onto github it is completely public and vulnerable to attack. Assume that if you put sensitive information publicly on the internet it will be found and abused. 

To this end, how can we easily access our API key without opening ourselves up to vulnerabilities?

There are many ways to store sensitive information but we will go with this method. 

#### Move to your home (root) directory:

```
cd ~
```

#### Now make the `.secret/` directory:

```
mkdir .secret
```

This will create a new folder in your home directory where you can store files for any of the API information you have. 

Can you find the file you just made in your terminal? 
NOTE: dot files won't show up with just `ls` you must use the show all command as well `ls -a`


#### Move into the newly created `.secret/` folder and create a file using vscode or any text editor to store your yelp API login info.

```
cd .secret/
code yelp_api.json
```

In this file, let's create a dictionary of values representing the client id and API key that looks something like this:

`{"api_key": "input api key here!"}`

NOTE: Double quotes are important! You'll copy and paste the `api_key` value that yelp grants you after you create your app.

Ok, so now we have a file in our .secret folder on our home directory. Safe and sound (mostly) from anyone trying to steal our info off github.

#### Finally, let's get our client id and API key into our jupyter notebook.

If we remember that our file is just a regular JSON file, open the file and pull out the appropriate information from the `~/.secret/yelp_api.json` file. 


In [1]:
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

> **Note**: Change the file path below to be your root directory. 
If you're not sure what your username is, check it with `pwd`  
For example, my current working directory is ```/Users/matthew.mitchell/Documents/dsc-using-yelp-api-codealong```  
So the line below would become:
```keys = get_keys("/Users/matthew.mitchell/.secret/yelp_api.json")```

In [3]:
keys = get_keys("/Users/joey/.secret/yelp_api.json")

api_key = keys['api_key']

#While you may wish to print out these API keys to check that they imported properly,
#be sure to clear the output before uploading to Github. 
#Again, you don't want your keys stolen!!!

## An Example Request with OAuth <a id="oauth_request"></a>
https://www.yelp.com/developers/documentation/v3/get_started

In the next lesson, we'll further dissect how to read and translate online documentation like the link here. For now, let's simply look at an example request and dissect it into its constituent parts:

In [11]:
import requests
term = 'Kosher'
location = 'Brooklyn NY'
SEARCH_LIMIT = 10

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "K2QvVZ1mXc1xZdA0NLlf9Q", "alias": "mama-kitchen-brooklyn", "name": "Mama Kitchen", "image_url": "https://s3-media4.fl.yelpcdn.com/bphoto/M30P6BIZIPcNwVCSz-aJDA/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/mama-kitchen-brooklyn?adjust_creative=jaEcwHv-qRlg5JJTpcsrDA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=jaEcwHv-qRlg5JJTpcsrDA", "review_count": 38, "categories": [{"alias": "kosher", "title": "Kosher"}, {"alias": "mideastern", "title": "Middle Eastern"}], "rating": 4.5, "coordinates": {"latitude": 40.6788993574647, "longitude": -73.9272566922721}, "transactions": ["pickup", "delivery"], "location": {"address1": "7 Rochester Ave", "address2": "", "address3": null, "city": "Brooklyn", "zip_code": "11233", "country": "US", "state": "NY", "display_address": ["7 Rochester Ave", "Brooklyn, NY 11233"]}, "phone": "+17187713038", "display_phone": "(718) 771-3038", "distance": 3125.799321047101}, 

## Breaking Down the Request

As you can see, there are three main parts to our request.  
  
They are:
* The URL
* The header
* The parameters
  
The URL is fairly straightforward and is simply the base URL as described in the documentation (again more details in the upcoming lesson).

The header is a dictionary of key-value pairs. In this case, we are using a fairly standard header used by many APIs. It has a strict form where 'Authorization' is the key and 'Bearer YourApiKey' is the value.

The parameters are the filters that we wish to pass into the query. These will be embedded into the URL when the request is made to the API. Similar to the header, they form key-value pairs. Valid key parameters by which to structure your queries are described in the API documentation which we'll look at further shortly. A final important note, however, is the need to replace spaces with "+". This is standard to many requests as URLs cannot contain spaces. (Note that the header itself isn't directly embedded into the URL itself and as such, the space between 'Bearer' and YourApiKey is valid.)


## The Response

As before, our response object has both a status code, as well as the data itself. With that, let's start with a little data exploration!

In [12]:
response.json().keys()

dict_keys(['businesses', 'total', 'region'])

Now let's go a bit further and start to preview what's stored in each of the values for these keys.

In [13]:
for key in response.json().keys():
    print(key)
    value = response.json()[key] #Use standard dictionary formatting
    print(type(value)) #What type is it?
    print('\n\n') #Separate out data

businesses
<class 'list'>



total
<class 'int'>



region
<class 'dict'>





Let's continue to preview these further to get a little better acquainted.

In [14]:
response.json()['businesses'][:2]

[{'id': 'K2QvVZ1mXc1xZdA0NLlf9Q',
  'alias': 'mama-kitchen-brooklyn',
  'name': 'Mama Kitchen',
  'image_url': 'https://s3-media4.fl.yelpcdn.com/bphoto/M30P6BIZIPcNwVCSz-aJDA/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/mama-kitchen-brooklyn?adjust_creative=jaEcwHv-qRlg5JJTpcsrDA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=jaEcwHv-qRlg5JJTpcsrDA',
  'review_count': 38,
  'categories': [{'alias': 'kosher', 'title': 'Kosher'},
   {'alias': 'mideastern', 'title': 'Middle Eastern'}],
  'rating': 4.5,
  'coordinates': {'latitude': 40.6788993574647,
   'longitude': -73.9272566922721},
  'transactions': ['pickup', 'delivery'],
  'location': {'address1': '7 Rochester Ave',
   'address2': '',
   'address3': None,
   'city': 'Brooklyn',
   'zip_code': '11233',
   'country': 'US',
   'state': 'NY',
   'display_address': ['7 Rochester Ave', 'Brooklyn, NY 11233']},
  'phone': '+17187713038',
  'display_phone': '(718) 771-3038',
  'distance': 3125.799321

In [15]:
response.json()['total']

2500

In [16]:
response.json()['region']

{'center': {'longitude': -73.93936157226562, 'latitude': 40.652330148320374}}

As you can see, we're primarily interested in the 'businesses' entry. 

Let's go ahead and create a dataframe from that.

In [18]:
import pandas as pd

df = pd.DataFrame.from_dict(response.json()['businesses'])
print(len(df)) #Print how many rows
print(df.columns) #Print column names
df.head(50) #Previews the first five rows. 
#You could also write df.head(10) to preview 10 rows or df.tail() to see the bottom

10
Index(['id', 'alias', 'name', 'image_url', 'is_closed', 'url', 'review_count',
       'categories', 'rating', 'coordinates', 'transactions', 'location',
       'phone', 'display_phone', 'distance', 'price'],
      dtype='object')


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,K2QvVZ1mXc1xZdA0NLlf9Q,mama-kitchen-brooklyn,Mama Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/M30P6B...,False,https://www.yelp.com/biz/mama-kitchen-brooklyn...,38,"[{'alias': 'kosher', 'title': 'Kosher'}, {'ali...",4.5,"{'latitude': 40.6788993574647, 'longitude': -7...","[pickup, delivery]","{'address1': '7 Rochester Ave', 'address2': ''...",+17187713038,(718) 771-3038,3125.799321,NaN
1,37RhWHnMsnjULuoN95Bg9A,izzys-brooklyn-smokehouse-crown-heights,Izzy's Brooklyn Smokehouse,https://s3-media3.fl.yelpcdn.com/bphoto/e5YYl_...,False,https://www.yelp.com/biz/izzys-brooklyn-smokeh...,144,"[{'alias': 'bbq', 'title': 'Barbeque'}, {'alia...",4.0,"{'latitude': 40.66487, 'longitude': -73.9369}","[pickup, delivery]","{'address1': '397 Troy Ave', 'address2': '', '...",+13474250524,(347) 425-0524,1409.865578,$$$
2,Xy5zmV2va1RTgnqyakfMnQ,gruit-brooklyn,Gruit,https://s3-media3.fl.yelpcdn.com/bphoto/WlHS8h...,False,https://www.yelp.com/biz/gruit-brooklyn?adjust...,12,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"{'latitude': 40.663465879123, 'longitude': -73...","[pickup, delivery]","{'address1': '252 Empire Blvd', 'address2': ''...",+13478460622,(347) 846-0622,1727.787787,NaN
3,lNV-bBLb9iTFCmbeGakwGw,gottliebs-restaurant-brooklyn-14,Gottlieb's Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/csYVkF...,False,https://www.yelp.com/biz/gottliebs-restaurant-...,67,"[{'alias': 'kosher', 'title': 'Kosher'}, {'ali...",3.5,"{'latitude': 40.7077128, 'longitude': -73.9611...","[pickup, delivery]","{'address1': '352 Roebling St', 'address2': ''...",+17183846612,(718) 384-6612,6426.086577,$$
4,dN17XjupSnAdIapJyXK_qg,brooklyn-artisan-bakehouse-and-cafe-brooklyn,Brooklyn Artisan Bakehouse & Cafe,https://s3-media1.fl.yelpcdn.com/bphoto/SiHnWu...,False,https://www.yelp.com/biz/brooklyn-artisan-bake...,52,"[{'alias': 'cafes', 'title': 'Cafes'}, {'alias...",4.0,"{'latitude': 40.6621475219727, 'longitude': -7...","[pickup, delivery]","{'address1': '529 E New York Ave', 'address2':...",+13472921382,(347) 292-1382,1188.967434,$$
5,pSAAr8srYYt4SUQP4r0Bhw,luannes-wild-ginger-brooklyn,Luanne's Wild Ginger,https://s3-media2.fl.yelpcdn.com/bphoto/_-6RMa...,False,https://www.yelp.com/biz/luannes-wild-ginger-b...,73,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 40.6896, 'longitude': -73.96982}","[pickup, delivery]","{'address1': '235 Dekalb Ave', 'address2': Non...",+17186388868,(718) 638-8868,4879.438731,$$
6,u2uujeuNdUA0WuF46xrsGg,meat-cafe-brooklyn-5,Meat Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/qmiiRj...,False,https://www.yelp.com/biz/meat-cafe-brooklyn-5?...,11,"[{'alias': 'kosher', 'title': 'Kosher'}]",4.0,"{'latitude': 40.67577, 'longitude': -73.94146}",[],"{'address1': '123 Kingston Ave', 'address2': N...",+15168881730,(516) 888-1730,2609.078224,NaN
7,-kywtQtjS4s6x2LVYxmreQ,grill-point-brooklyn-brooklyn-3,Grill Point - Brooklyn,https://s3-media3.fl.yelpcdn.com/bphoto/HhPSmk...,False,https://www.yelp.com/biz/grill-point-brooklyn-...,8,"[{'alias': 'mideastern', 'title': 'Middle East...",4.5,"{'latitude': 40.60807, 'longitude': -73.96114}","[pickup, delivery]","{'address1': '1123 Quentin Rd', 'address2': No...",+17187669009,(718) 766-9009,5244.943538,NaN
8,Kg1OqaEPNFFL5FuqnALdWQ,davids-brisket-house-and-deli-brooklyn,David's Brisket House and Deli,https://s3-media2.fl.yelpcdn.com/bphoto/1zevDw...,False,https://www.yelp.com/biz/davids-brisket-house-...,437,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",4.5,"{'latitude': 40.679434, 'longitude': -73.949515}","[pickup, delivery]","{'address1': '533 Nostrand Ave', 'address2': '...",+17187891155,(718) 789-1155,3133.116516,$$
9,x3WjQRDbbyqaWZ4ABgUX5Q,essen-ny-deli-brooklyn,Essen NY Deli,https://s3-media4.fl.yelpcdn.com/bphoto/Y4RvBY...,False,https://www.yelp.com/biz/essen-ny-deli-brookly...,74,"[{'alias': 'delis', 'title': 'Delis'}, {

In [ ]:
# Got kosher restaurants in brooklyn!

## Summary <a id="sum"></a>

Congratulations! We've covered a lot here! We took some of your previous knowledge with HTTP requests and OAuth in order to leverage an enterprise API! Then we made some requests to retrieve information that came back as a JSON format. We then transformed this data into a dataframe using the Pandas package. In the next lab, we'll break down how to read API documentation!